# Forecasting SIRDS time series for the epidemic episodes

In [1]:

import pandas as pd
from scipy.optimize import differential_evolution
from datetime import datetime

from code.sirds.sirds_model import get_bounds_and_arguments, sirds_objective_function

## Reading data

In [2]:
df = pd.read_csv('../data/df_original_extended.csv')
df.DATA = pd.to_datetime(df.DATA, format='mixed')

In [12]:
df[df['DATA'] == df[df['NOVOS_CASOS_SRAG']>0]['DATA'].min()]

,DATA,CODIGO_MUNICIPIO_6,NOVOS_CASOS_SRAG,OBITOS_NOVOS,OBITOS,OBITOS_NOVOS_MEDIA_MOVEL_7_DIAS,TAXA_OBITOS_NOVOS,TAXA_OBITOS_NOVOS_MEDIA_MOVEL_7_DIAS,TAXA_OBITOS,casosNovos,...,NUMERO_REPRODUCAO_EFETIVO_QUANTIL_0.975,NUMERO_REPRODUCAO_EFETIVO_ATRASADO_MEDIA,NUMERO_REPRODUCAO_EFETIVO_ATRASADO_VARIANCIA,NUMERO_REPRODUCAO_EFETIVO_ATRASADO_QUANTIL_0.025,NUMERO_REPRODUCAO_EFETIVO_ATRASADO_MEDIANA,NUMERO_REPRODUCAO_EFETIVO_ATRASADO_QUANTIL_0.975,TAXA_FATALIDADE,EPIDEMIC_EPISODE,ONSET_NUMERO_REPRODUCAO_EFETIVO_MEDIA,ONSET_TAXA_OBITOS_NOVOS_MEDIA_MOVEL_7_DIAS
0,2020-02-16,520140,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
574,2020-02-16,280030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1148,2020-02-16,310620,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1722,2020-02-16,150140,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2296,2020-02-16,530010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2870,2020-02-16,350950,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3444,2020-02-16,500270,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4018,2020-02-16,311860,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4592,2020-02-16,510340,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5166,2020-02-16,410690,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df[df['MUNICIPIO']=='São Paulo'][['DATA','MUNICIPIO','casosAcumulado']]

,DATA,MUNICIPIO,casosAcumulado
21812,2020-02-16,São Paulo,0.0
21813,2020-02-17,São Paulo,0.0
21814,2020-02-18,São Paulo,0.0
21815,2020-02-19,São Paulo,0.0
21816,2020-02-20,São Paulo,0.0
...,...,...,...
33357,2022-05-17,São Paulo,1060504.0
33358,2022-05-18,São Paulo,1060846.0
33359,2022-05-19,São Paulo,1060916.0
33360,2022-05-20,São Paulo,1060916.0


In [4]:
cumulative_days_in_first_outbreak_to_max_bound_I0 = 56

## Forecasting

In [5]:
df = df.sort_values(by=['MUNICIPIO','DATA'])

In [6]:
df.columns

In [7]:
df['TAXA_CASOS_NOVOS_MEDIA_MOVEL_7_DIAS'] = df['CASOS_NOVOS_MEDIA_MOVEL_7_DIAS'] / df['POPULACAO_2022'] * 100000

In [8]:
list_municipalities = df['MUNICIPIO'].unique()
list_municipalities_complete = []
list_municipalities_to_complete = list(set(list_municipalities) - set(list_municipalities_complete))

In [9]:
epidemiologic_weeks = [pd.to_datetime('2020-04-26'), pd.to_datetime('2020-07-19'), pd.to_datetime('2020-10-11'), pd.to_datetime('2021-01-03'), pd.to_datetime('2021-03-28'), pd.to_datetime('2021-06-20'), pd.to_datetime('2021-09-12'), pd.to_datetime('2021-12-05'), pd.to_datetime(('2022-02-27'))]

In [ ]:
try:
    df_initial_results = pd.read_csv('data/estimation_results.csv')
    df_initial_results['max_date_to_fit'] = pd.to_datetime(df_initial_results['max_date_to_fit'])
except:
    df_initial_results = pd.DataFrame({'municipality_id':[], 'max_date_to_fit':[]})
    
df_results = df_initial_results.copy()

DAYS_TO_RECOVERY = 8
DIFFERENTIAL_EVOLUTION_POP_SIZE_FACTOR = 5
NUMBER_ESTIMATON_PER_MUNICIPALITY = 20

for estimation in range(NUMBER_ESTIMATON_PER_MUNICIPALITY):
    print('estimation: ', estimation)
    for week_begin in epidemiologic_weeks:
        max_date_to_fit = week_begin - pd.DateOffset(days=1)
        print('max_date_to_fit: ', max_date_to_fit)
        for municipality in df[df['MUNICIPIO'].isin(list_municipalities_to_complete)]['MUNICIPIO'].unique():
            print('municipality: ', municipality)
            df_municipality = df[(df.MUNICIPIO == municipality) & (df.DATA <= max_date_to_fit)].reset_index(drop=True)
            municipality_id = df_municipality.CODIGO_MUNICIPIO_6.unique()[0]
            
            # Dates about begin time series and first case      
            date_first_case = min(df_municipality[(df_municipality.NOVOS_CASOS_SRAG > 0)].iloc[0].DATA, df_municipality[(df_municipality.casosNovos > 0)].iloc[0].DATA)       
            # Period of analysis
            period_in_days = (max_date_to_fit - date_first_case).days + 1
            
            estimations_performed = len(df_results[((df_results.municipality_id == municipality_id) & (df_results.max_date_to_fit == max_date_to_fit))])
     
            if (estimation == estimations_performed):
                print('Municipality: ', municipality)
                print('max_date_to_fit', max_date_to_fit)
                print('estimation: ', estimation)
                
                rt_column = 'NUMERO_REPRODUCAO_EFETIVO_SRAG_MEDIA'
                
                count_rt_items = len(df_municipality[rt_column].dropna())
                
                if count_rt_items >= 7:
                    bounds, args = get_bounds_and_arguments(df_municipality, 'DATA', 'TAXA_OBITOS_NOVOS_MEDIA_MOVEL_7_DIAS', rt_column, 'TAXA_CASOS_NOVOS_MEDIA_MOVEL_7_DIAS', 'ONSET_NUMERO_REPRODUCAO_EFETIVO_MEDIA', DAYS_TO_RECOVERY, date_first_case, max_date_to_fit, df_municipality.POPULACAO_2022.iloc[0], period_in_days, cumulative_days_in_first_outbreak_to_max_bound_I0)
                    
                    print(bounds)
                    
                    # Record the start time
                    start_time = datetime.now()
                    print(start_time)
        
                    result = differential_evolution(sirds_objective_function, bounds, args=args, popsize=DIFFERENTIAL_EVOLUTION_POP_SIZE_FACTOR, maxiter=10000, workers=11, updating='deferred')
            
                    # Record the end time
                    end_time = datetime.now()
            
                    # Calculate the duration (in seconds) for the optimization
                    duration = (end_time - start_time).total_seconds()
                    print(duration)
                    
                    list_breakpoints_in_slow_transition = args[4]
                    quantity_outbreaks = args[5]
                    quantity_outbreak_adjustments = args[6]
            
                    # Create a dictionary to store results
                    estimation_result = {
                        'municipality_id': municipality_id,
                        'municipality': municipality,
                        'max_date_to_fit': max_date_to_fit,
                        'estimation': estimation,  # To differentiate between multiple estimations
                        'result_fun': result.fun,
                        'result_nfev': result.nfev,
                        'result_nit': result.nit,
                        'result_success': result.success,
                        'start_time': start_time.strftime('%Y-%m-%d %H:%M:%S'),  # Format start time as a string
                        'end_time': end_time.strftime('%Y-%m-%d %H:%M:%S'),  # Format end time as a string
                        'duration_seconds': duration,  # Duration in seconds
                        'pop_size': DIFFERENTIAL_EVOLUTION_POP_SIZE_FACTOR,
                        'period_in_days': period_in_days,
                        'days_to_recovery': DAYS_TO_RECOVERY,
                        'cumulative_days_in_first_outbreak_to_max_bound_I0': cumulative_days_in_first_outbreak_to_max_bound_I0,
                        'date_first_case': date_first_case,
                        'list_breakpoints_in_slow_transition': list_breakpoints_in_slow_transition,
                        'x_initial_infected_population': result.x[0],
                        'x_days_between_infections_0': result.x[1]
                    }
                    
                    quantity_epidemic_periods_with_slow_transition = len(list_breakpoints_in_slow_transition) + 1
                    
                    for p in range(quantity_epidemic_periods_with_slow_transition):
                        estimation_result['x_case_fatality_probability_'+str(p)] = result.x[2 + p]
                        estimation_result['x_loss_immunity_in_days_'+str(p)] = result.x[2 + quantity_epidemic_periods_with_slow_transition + p]                                                 
                    quantity_breakpoints = (quantity_outbreaks - 1) + (quantity_outbreak_adjustments)
                    begin_breakpoint_parameters = 2 + 2*quantity_epidemic_periods_with_slow_transition
                    for b in range(quantity_breakpoints):
                        estimation_result['x_days_between_infections_'+str(b+1)] = result.x[begin_breakpoint_parameters + b]
                        estimation_result['x_breakpoint_'+str(b+1)] = result.x[begin_breakpoint_parameters + quantity_breakpoints + b]
                        estimation_result['x_transition_days_between_epidemic_periods_'+str(b+1)] = result.x[begin_breakpoint_parameters + 2*quantity_breakpoints + b]
            
                    print(estimation_result)
            
                    # Append the estimation result to the list            
                    df_results = pd.concat([df_results, pd.DataFrame.from_records([estimation_result])])
            
                    df_results.to_csv('data/estimation_results.csv', index=False) 

In [ ]:
from matplotlib import pyplot as plt

plt.plot(df_municipality[rt_column])

In [ ]:
df_municipality[rt_column].dropna().max()

In [ ]:
8/df_municipality[rt_column].dropna().max()

In [ ]:
df_municipality[rt_column].dropna().mean()

In [ ]:
df_municipality[rt_column].dropna().iloc[-1:].quantile(0.25)

In [ ]:
df_municipality[rt_column]